In [1]:
import mmap
import os
import random
import struct
import tempfile


REC_SIZE = struct.calcsize('I') # our items is unsigned integers (4 bytes)

# perform binary search using read() from file descriptor
#   for each value
def bsearch_reads(fd, n, x):
    l, h = 0, n

    while l < h:
        mid = (l + h) // 2
        
        # read single integer from file
        fd.seek(mid * REC_SIZE)
        data = fd.read(REC_SIZE)
        
        val = struct.unpack('I', data)[0] #  The result is a tuple even if it contains exactly one item
        if x == val:
            return True
        if x < val:
            h = mid
        else:
            l = mid + 1

    return False


# perform binary within string 's' of binary data
def bsearch_mmap(s, n, x):
    l, h = 0, n

    while l < h:
        mid = (l + h) // 2
        val = struct.unpack_from('I', s, mid*REC_SIZE)[0] #  The result is a tuple even if it contains exactly one item
        if x == val:
            return True
        if x < val:
            h = mid
        else:
            l = mid + 1

    return False



NNUMS = 10**5
NUM_SPACE = NNUMS * 3

nums = sorted(random.sample(xrange(NUM_SPACE), NNUMS))

fd = tempfile.TemporaryFile()

for x in nums:
    fd.write(struct.pack('I', x))

fd.seek(0)

# mmap() our temporary file
mem = mmap.mmap(fd.fileno(), 0, prot=mmap.PROT_READ)
nfound = 0

for _ in range(100):
    x = random.randint(0, NUM_SPACE)
    found_read = bsearch_reads(fd, len(nums), x)
    found_mmap = bsearch_mmap(mem, len(nums), x)
    if found_read != found_mmap:
        raise RuntimeError("bsearch-es not be equal!")
    if found_read:
        nfound += 1
        
print 'That should work! nfound=%d' % nfound

That should work! nfound=31
